<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/294_HITL_Routing_utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Routing decision utilities for HITL Orchestrator

In [ ]:
"""Routing decision utilities for HITL Orchestrator"""

from typing import Dict, Any, Optional, List


def apply_routing_policy(
    risk_level: str,
    confidence_score: float,
    routing_policy: Dict[str, Any]
) -> Dict[str, Any]:
    """
    Apply routing policy to determine routing decision for a task.

    Args:
        risk_level: Task risk level ("low", "medium", "high")
        confidence_score: Agent confidence score (0.0-1.0)
        routing_policy: Routing policy dictionary with rules

    Returns:
        Routing decision dictionary with:
        - routing_decision: "auto_approve" | "human_review" | "escalate"
        - assigned_human_role: Optional[str]
        - rule_applied: str (rule_id)
        - reasoning: str
    """
    rules = routing_policy.get("rules", [])

    # Sort rules by priority (lower number = higher priority)
    sorted_rules = sorted(rules, key=lambda r: r.get("priority", 999))

    # Find first matching rule
    for rule in sorted_rules:
        conditions = rule.get("conditions", {})

        # Check risk level condition
        if "risk_level" in conditions:
            if risk_level != conditions["risk_level"]:
                continue

        # Check confidence condition
        if "min_confidence" in conditions:
            min_confidence = conditions["min_confidence"]
            if confidence_score < min_confidence:
                continue

        # Rule matches - apply it
        action = rule.get("action")
        assigned_role = rule.get("assigned_human_role")
        rule_id = rule.get("rule_id", "unknown")

        # Build reasoning
        reasoning_parts = []
        if "risk_level" in conditions:
            reasoning_parts.append(f"Risk level: {risk_level}")
        if "min_confidence" in conditions:
            reasoning_parts.append(f"Confidence: {confidence_score:.2f} >= {conditions['min_confidence']}")
        reasoning = ", ".join(reasoning_parts) if reasoning_parts else "Default rule"

        return {
            "routing_decision": action,
            "assigned_human_role": assigned_role,
            "rule_applied": rule_id,
            "reasoning": reasoning
        }

    # No rule matched - default to human review
    return {
        "routing_decision": "human_review",
        "assigned_human_role": "domain_reviewer",
        "rule_applied": "default",
        "reasoning": "No matching rule found, defaulting to human review"
    }


def make_routing_decision(
    task_id: str,
    risk_level: str,
    confidence_score: float,
    routing_policy: Dict[str, Any]
) -> Dict[str, Any]:
    """
    Make routing decision for a single task.

    Args:
        task_id: Task identifier
        risk_level: Task risk level
        confidence_score: Agent confidence score
        routing_policy: Routing policy

    Returns:
        Complete routing decision dictionary
    """
    decision = apply_routing_policy(risk_level, confidence_score, routing_policy)

    return {
        "task_id": task_id,
        "risk_level": risk_level,
        "confidence_score": confidence_score,
        **decision
    }





# 🧠 Big Picture: What This Code Does

This file answers **one single question**:

> **“Given a task’s risk and the AI’s confidence, who should decide?”**

That’s it.

Everything else in the orchestrator exists to *support* this moment.

---

# 🧩 Mental Model (Very Important)

Think of the routing policy as a **rulebook** and this code as a **referee**.

* The rulebook says *what should happen*
* The referee checks the rules **in order**
* The referee stops at the **first rule that applies**
* The referee explains the decision

No guessing. No learning. No magic.

---

# Part 1: `apply_routing_policy`

## 🎯 “Read the rules and pick one”

```python
def apply_routing_policy(...)
```

### What this function is responsible for

This function:

* does **NOT** know about task IDs
* does **NOT** know about humans yet
* does **NOT** finalize anything

It only answers:

> “Based on risk and confidence, what does the policy say?”

---

## 🔢 Step 1: Get the rules

```python
rules = routing_policy.get("rules", [])
```

Conceptually:

* Pull the list of rules out of the policy
* If there are none, use an empty list (fail safely)

---

## 🥇 Step 2: Sort rules by priority

```python
sorted_rules = sorted(...)
```

### Why this matters conceptually

Rules are **not equal**.

Some are more important:

* “High risk → escalate” should beat everything
* “Low risk + high confidence → auto” comes later

This sorting guarantees:

> **More serious rules are checked first**

This avoids accidental automation.

---

## 🔍 Step 3: Check rules one by one

```python
for rule in sorted_rules:
```

This is like reading a checklist:

> “Does this rule apply?
> No? Next.
> Yes? Stop.”

The first match wins.

---

## ⚠️ Step 4: Check risk level

```python
if risk_level != conditions["risk_level"]:
    continue
```

Plain English:

* If the rule cares about risk
* And the task’s risk doesn’t match
* Skip the rule

This keeps rules **specific and safe**.

---

## 📊 Step 5: Check confidence

```python
if confidence_score < min_confidence:
    continue
```

Plain English:

* If the rule requires confidence
* And the AI isn’t confident enough
* Skip the rule

This is the **core safety mechanism**.

---

## ✅ Step 6: Apply the rule

Once both checks pass:

```python
return {
  routing_decision,
  assigned_human_role,
  rule_applied,
  reasoning
}
```

### Why this return is powerful

You’re not just saying *what* happened.
You’re also saying:

* which rule fired
* why it fired

That’s **explainable AI without AI**.

---

## 🧠 The “Reasoning” field (very important)

This line:

```python
reasoning = "Risk level: low, Confidence: 0.91 >= 0.80"
```

Exists for **humans**, not machines.

It answers:

> “Why did the system do this?”

This is how trust is built.

---

## 🚨 Fallback: No rule matched

```python
return human_review
```

Conceptually:

> **When in doubt, ask a human.**

This is the single most important safety principle in the entire system.

---

# Part 2: `make_routing_decision`

## 🧾 “Wrap it up with context”

```python
def make_routing_decision(...)
```

This function:

* calls the rule engine
* attaches task identity
* produces a complete decision record

Think of it as:

* the referee making the call
* then writing it down on the scoreboard

---

## 🧠 Why separate these two functions?

* `apply_routing_policy` = **pure logic**
* `make_routing_decision` = **system integration**

This separation makes your system:

* testable
* reusable
* understandable

Good agents are built from **small, honest parts**.

---

# 🎯 Big Takeaway (Most Important)

This code is teaching you a deep idea:

> **Autonomy is not about intelligence — it’s about permission.**

The AI doesn’t decide because it’s “smart”.
It decides because:

* risk is acceptable
* confidence is high
* rules allow it

That’s how real-world AI works.

